# Labels Data Preprocessing

In [1]:
import time
import matplotlib.pyplot as plt
from skimage import exposure,img_as_float
import cv2, os
import numpy as np
import pydicom
import pandas as pd
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

In [2]:
csv_dir =os.path.dirname(os.getcwd())+'/CT_Project'
csv_dir

'/scratch/msk759/msk759/DL/CT_Project'

In [3]:
labels0_df = pd.read_excel("TCIA CTC no polyp found.xls")

labels0_df.rename(columns={'TCIA Patient ID':'TCIA Number'}, inplace=True)

labels1_df = pd.read_excel("TCIA CTC 6 to 9 mm polyps.xls")


labels2_df = pd.read_excel("TCIA CTC large 10 mm polyps.xls")


labels_df = pd.concat([labels0_df, labels1_df, labels2_df], axis=0, ignore_index=True)

In [4]:
labels_df = labels_df.iloc[:,[0,7]]
labels_df.fillna(0, inplace=True)
labels_df.rename(columns={1.5:'Label'}, inplace=True)
labels_df

,TCIA Number,Label
0,1.3.6.1.4.1.9328.50.4.0001,0.0
1,1.3.6.1.4.1.9328.50.4.0003,0.0
2,1.3.6.1.4.1.9328.50.4.0004,0.0
3,1.3.6.1.4.1.9328.50.4.0005,0.0
4,1.3.6.1.4.1.9328.50.4.0006,0.0
5,1.3.6.1.4.1.9328.50.4.0009,0.0
6,1.3.6.1.4.1.9328.50.4.0010,0.0
7,1.3.6.1.4.1.9328.50.4.0011,0.0
8,1.3.6.1.4.1.9328.50.4.0012,0.0
9,1.3.6.1.4.1.9328.50.4.0013,0.0


In [5]:
# Get names of indexes for which column Age has value 16 98 88
indexNames = labels_df[(labels_df['Label'] == 16.0 )|(labels_df['Label'] == 88.0 )|(labels_df['Label'] == 98.0 )].index
# Delete these row indexes from dataFrame
labels_df.drop(indexNames , inplace=True)

labels_df['Class'] = np.where(labels_df['Label']==0.0
                              ,0,1)
labels_df

,TCIA Number,Label,Class
0,1.3.6.1.4.1.9328.50.4.0001,0.0,0
1,1.3.6.1.4.1.9328.50.4.0003,0.0,0
2,1.3.6.1.4.1.9328.50.4.0004,0.0,0
3,1.3.6.1.4.1.9328.50.4.0005,0.0,0
4,1.3.6.1.4.1.9328.50.4.0006,0.0,0
5,1.3.6.1.4.1.9328.50.4.0009,0.0,0
6,1.3.6.1.4.1.9328.50.4.0010,0.0,0
7,1.3.6.1.4.1.9328.50.4.0011,0.0,0
8,1.3.6.1.4.1.9328.50.4.0012,0.0,0
9,1.3.6.1.4.1.9328.50.4.0013,0.0,0


In [6]:
# Only include some for experimental use
####################################
# CT>10
# 1.3.6.1.4.1.9328.50.4.0040 
# 
# CT(6<polyp)
# 1.3.6.1.4.1.9328.50.4.0007 
# 
# 1.3.6.1.4.1.9328.50.4.0019 
# 1.3.6.1.4.1.9328.50.4.0043 
# 1.3.6.1.4.1.9328.50.4.0080 
# 
# CT(no polyp)
# 1.3.6.1.4.1.9328.50.4.0001 
# 1.3.6.1.4.1.9328.50.4.0003 
# 1.3.6.1.4.1.9328.50.4.0004 
# 1.3.6.1.4.1.9328.50.4.0005 
# 1.3.6.1.4.1.9328.50.4.0006 
####################################
experimentals=[
    "1.3.6.1.4.1.9328.50.4.0040",
    "1.3.6.1.4.1.9328.50.4.0007",
    "1.3.6.1.4.1.9328.50.4.0019",
    "1.3.6.1.4.1.9328.50.4.0043",
    "1.3.6.1.4.1.9328.50.4.0080",
    "1.3.6.1.4.1.9328.50.4.0001",
    "1.3.6.1.4.1.9328.50.4.0003",
    "1.3.6.1.4.1.9328.50.4.0004",
    "1.3.6.1.4.1.9328.50.4.0005",
    "1.3.6.1.4.1.9328.50.4.0006"]

In [7]:
labels_df_exp= labels_df.loc[labels_df['TCIA Number'].isin(experimentals)]
labels_df_exp

,TCIA Number,Label,Class
0,1.3.6.1.4.1.9328.50.4.0001,0.0,0
1,1.3.6.1.4.1.9328.50.4.0003,0.0,0
2,1.3.6.1.4.1.9328.50.4.0004,0.0,0
3,1.3.6.1.4.1.9328.50.4.0005,0.0,0
4,1.3.6.1.4.1.9328.50.4.0006,0.0,0
243,1.3.6.1.4.1.9328.50.4.0007,10.0,1
245,1.3.6.1.4.1.9328.50.4.0019,13.0,1
246,1.3.6.1.4.1.9328.50.4.0043,17.0,1
247,1.3.6.1.4.1.9328.50.4.0080,10.0,1
312,1.3.6.1.4.1.9328.50.4.0040,1.0,1


# DICOM Data Processing


In [47]:
data_dir = csv_dir+'/CT_Project'
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        filepath = subdir + os.sep + file
        foldername = subdir.split(os.sep)[-1]

        number = ""
        foundnumber = False

        for c in filepath:
            if c.isdigit():
                foundnumber = True
                number = number + c
            elif foundnumber:
                break
        if foundnumber:
            newfilepath = filepath.replace(number,foldername)
            os.rename(filepath, newfilepath)

# Making new file names

In [69]:
data_dir= csv_dir+'/CT COLONOGRAPHY'
for root, dirs, files in os.walk(data_dir, topdown=False):
    for file in files:
        file_name = os.path.splitext(file)[0]#file name no ext
        extension = os.path.splitext(file)[1]
        dir_name = os.path.basename(root)
        os.rename(root+"/"+file,root+"/"+data_dir+extension)

In [68]:
replace(csv_dir,data_dir,'NEW')

In [64]:
for root,dirname,filenames in os.walk(old):   
    for filename in filenames:
        i = filename.split(".")
        print(last+"_"+i[0]+"_"+root.split("/")[-1]+"."+i[1])
        os.rename(os.path.join(rootrenm, filename),"{last}_{start}_{sub_fold}.{ext}".format(last=last, start=i[0], sub_fold=root.split("/")[-1], ext=i[1]))

In [21]:
data_dir=csv_dir+'/data/'
patients = os.listdir(data_dir)
output_path = working_path = "/Users/MOOSE/Desktop/DL/CT_Project/"
g = glob(data_dir + '/*.dcm')

# Print out the first 5 file names to verify we're in the right folder.
print ("Total of %d DICOM images.\nFirst 5 filenames:" % len(g))
print ('\n'.join(g[:5]))

Total of 625 DICOM images.
First 5 filenames:
/scratch/msk759/msk759/DL/CT_Project/data/000082.dcm
/scratch/msk759/msk759/DL/CT_Project/data/000202.dcm
/scratch/msk759/msk759/DL/CT_Project/data/000195.dcm
/scratch/msk759/msk759/DL/CT_Project/data/000579.dcm
/scratch/msk759/msk759/DL/CT_Project/data/000210.dcm


In [38]:
file_paths = {}
for root, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('.dcm'):
            if f not in file_paths:
                file_paths[f] = []
            file_paths[f].append(root)

In [44]:
ds =pydicom.dcmread(g[0])
ds

(0008, 0000) Group Length                        UL: 446
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.9328.50.4.781
(0008, 0020) Study Date                          DA: '20000101'
(0008, 0021) Series Date                         DA: '20000101'
(0008, 0022) Acquisition Date                    DA: '20000101'
(0008, 0023) Content Date                        DA: '20000101'
(0008, 0030) Study Time                          TM: '075555.562000'
(0008, 0032) Acquisition Time                    TM: '081509.823081'
(0008, 0033) Content Time                        TM: '081509.823081'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'CT'
(0008, 0070) Manufacturer                        L

In [33]:
for i in range(len(g)):
    ds = pydicom.dcmread(g[i])  # plan dataset
    print(ds.PatientName)

1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1

1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1.3.6.1.4.1.9328.50.4.0001
1

In [14]:
def readScan(fileList, imgType='DICOM', new_spacing=[5,1,1]):
    #Read a DICOM stack - fileList is a directory
    if imgType == 'DICOM':
        # Read in all of the slices
        slices = [pydicom.read_file(s) 
            for s in glob(fileList + '/*.dcm')]
        slices.sort(key = lambda x: int(x.InstanceNumber))
        
        #Get the slice thickness and pixel spacing
        try:
            slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - 
                                     slices[1].ImagePositionPatient[2])
        except:
            slice_thickness = np.abs(slices[0].SliceLocation - 
                                     slices[1].SliceLocation)
        
        pixel_spacing = slices[0].PixelSpacing
        
        
        # Get the slope and intercept
        intercept = slices[0].RescaleIntercept
        slope = slices[0].RescaleSlope
        
        # Make them a numpy array
        image = np.stack([s.pixel_array for s in slices])
        image = image.astype(np.int16)
        
        # Set outside-of-scan pixels to air
        # The intercept is usually -1024, so air is approximately 0
        image[image < 0] = 0

        #Convert to Hounsfield units (HU)
        if slope != 1:
            image = slope * image.astype(np.float64)
            image = image.astype(np.int16)

        image += np.int16(intercept)

    #Read a NIFTI image - fileList is a file
#     elif imgType == 'NIFTI':
#         niftiData = nifti.load(fileList)
        
#         #Get the slice thickness and pixel spacing
#         slice_thickness = niftiData.header.get_zooms()[2]
#         pixel_spacing = list(niftiData.header.get_zooms()[:2])
        
#         # Get the slope and intercept
#         intercept = niftiData.dataobj.inter
#         slope = niftiData.dataobj.slope
        
#         #Get the actual image data
#         image = np.array(niftiData.dataobj)
#         #Shift the axis order to match what we expect (z,x,y) and
#         #adjust to (0,0) in the upper right
#         image = np.rollaxis(image, 2, 0)
#         image = np.fliplr(np.flipud(np.swapaxes(image,1,2)))

    # Calculate the desired resampling
    spacing = map(float, [slice_thickness] + list(pixel_spacing))
    spacing = np.array(list(spacing))
#     spacing = map(float, ([scan[0].SliceThickness] + list(scan[0].PixelSpacing))) 
#     spacing = np.array(list(map(float, ([scan[0].SliceThickness] + list(scan[0].PixelSpacing)))))
    resize_factor = spacing / new_spacing
    new_shape = np.round(image.shape * resize_factor)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    return scipy.ndimage.interpolation.zoom(image, real_resize_factor)

In [15]:
dataDir = '/Users/MOOSE/Desktop/DL/CT_Project/data'
outpath = "/Users/MOOSE/Desktop/DL/CT_Project/"
scan = readScan(dataDir, imgType='DICOM')
#processedScan = segmentationCrop(scan)
#np.savez_compressed(outpath + '/Image.npz', data=processedScan)

IndexError: list index out of range

In [ ]:
scan.shape

In [ ]:
data_dir = '/Users/MOOSE/Desktop/DL/CT_Project/data'
patients = os.listdir(data_dir)

for patient in patients[:1]:
    label = labels_df.get_value(patient, 'Label')
    path = data_dir + patient
    
    # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    print(len(slices),label)
    print(slices[0])

In [ ]:
labels3_df.columns

In [ ]:
def dicom_2png(file):
    _currFile = file
    dcm = pydicom.dcmread(file)
    fileName = os.path.basename(file)
    imageX = dcm.pixel_array
    temp = imageX.copy()
    print("shape ----", imageX.shape)
    picMax = imageX.max()
    vmin = imageX.min()
    vmax = temp[temp < picMax].max()
    # print("vmin : ", vmin)
    # print("vmax : ", vmax)
    imageX[imageX > vmax] = 0
    imageX[imageX < vmin] = 0
    # result = exposure.is_low_contrast(imageX)
    # # print(result)
    image = img_as_float(imageX)
    plt.cla()
    plt.figure('adjust_gamma', figsize=(10.24, 10.24))
    plt.subplots_adjust(top=1, bottom=0, left=0, right=1, hspace=0, wspace=0)
    plt.imshow(image, 'gray')
    plt.axis('off')
    plt.savefig(fileName + '.png')
    time.sleep(1)


In [ ]:
dicom_2png('./000000.dcm')

In [ ]:
from skimage import exposure
import os
import pydicom
import math
import random
import numpy
import matplotlib.pyplot as plt
import time

def imageCount(dirName):
    """Counts the number of files in a directory
    Counts the number of files in a given directory and returns the result. Ignores desktop.ini to allow for Windows compatibility
    Args:
        dirName: A string representation of the name of the directory to be opened. This directory should be a child of the working directory. EX: "trainingImages"
    Returns:
        An integer value for the number of files in the directory
    """

    for root, dir, files in os.walk("./" + dirName):
        scans = [file  for file in files if file != "desktop.ini"]
        return len(scans)
    print(dirName + " is not a valid directory")
    exit(-1)

def readScan(scanNum, dirName):
    """ Reads a DICOM file from a directory
    Given a directory name and the number of the file containing the image to be read, reads in and returns a numpy representation of the image. Also normalizes the pixel values of the image.
    Args:
        scanNum: An integer containing the one-based index of the file to be read
        dirName: A string representation of the name of the directory to be opened. This directory should be a child of the working directory. Ex: "trainingImages"
    Returns: 
        A numpy array containing the pixel data for the image
    """

    for root, dir, files in os.walk("./" + dirName):
        scans = [file  for file in files if file != "desktop.ini"]
        print("Reading image " + scans[scanNum] + " from " + dirName)
        data = numpy.load("./" + dirName + "/" + scans[scanNum])
        #plt.imshow(data)
        #plt.show()
        data = exposure.equalize_adapthist(data)        
        #plt.imshow(data)
        #plt.show()
        return data
    print(dirName + " is not a valid directory")
    exit(-1)

def readValidationImages():
    """ Reads in all images in the validation directories
    Creates a numpy array containing the pixel data of all the images in the directories of validation images. Assumes that the subdirectories "PosVal" and "NegVal" exit in the working directory
    
    Args: None
    Returns: 
        A numpy array containing the pixel values for all validation images. Also returns a numpy array containing whether each image has a positive or negative label
    """
    
    labels = []
    patientData = []
    
    for i in range(0,imageCount("PosVal")):
        data = readScan(i, "PosVal")
        labels.append(1)
        patientData.append(data)
    for i in range(0,imageCount("NegVal")):
        data = readScan(i, "NegVal")
        labels.append(0)
        patientData.append(data)

    patientData = numpy.stack(patientData).astype(float)

    return patientData, numpy.stack(labels)

def readBatch(batchSize):
    """ Reads in a batch of images
    Given a batch size, reads a group of images at random containing an eqaul number of positive and negative examples. Assumes the directories "posTrain" and "negTrain" exist
    Args: 
        batchSize: An integer containing the total number of images to read. batchSize // 2 images will be read from the positive and negative training directories
    Returns:
        A numpy array containing the pixel values for the batch of images. Also returns a numpy array containing whether each image has a positive or negative label
    """

    labels = []
    patientData = []
    
    posLen = imageCount("PosTrain")
    negLen = imageCount("NegTrain")

    posStart = random.randint(0,(posLen- (batchSize//2)) - 1)
    negStart = random.randint(0,(negLen- (batchSize//2)) - 1)

    for i in range(batchSize //2):
        data = readScan(posStart + i, "PosTrain")
        labels.append(1)
        patientData.append(data)
        data = readScan(negStart + i, "NegTrain")
        labels.append(0)
        patientData.append(data)
    
    patientData = numpy.stack(patientData).astype(float)
    return patientData, numpy.stack(labels)

def normReadAll():
    """ Reads all images in the training set
    Reads all images from the training set. Assumes that the working directory has subdirectories "PosTrain" and "NegTrain"
    Returns:
        A numpy array containing the pixel values for the batch of images. Also returns a numpy array containing whether each image has a positive or negative label
    """
    
    labels = []
    patientData = []
    
    posLen = imageCount("PosTrain")
    negLen = imageCount("NegTrain")

    for i in range(posLen):
        data = readScan(i, "PosTrain")
        labels.append(1)
        data = exposure.equalize_adapthist(data)        
        patientData.append(data)
    for i in range(negLen):
        data = readScan(i, "NegTrain")
        labels.append(0)
        data = exposure.equalize_adapthist(data)        
        patientData.append(data)
    
    patientData = numpy.stack(patientData).astype(float)
    return patientData, numpy.stack(labels)

def readTest():
    """ Reads all images in the test set
    Reads all images from the test set. Assumes that the working directory has subdirectories "PosTest" and "NegTest"
    Returns:
        A numpy array containing the pixel values for the batch of images. Also returns a numpy array containing whether each image has a positive or negative label
    """
    labels = []
    patientData = []
    
    posLen = imageCount("PosTest")
    negLen = imageCount("NegTest")

    for i in range(posLen):
        data = readScan(i, "PosTest")
        labels.append(1)
        data = exposure.equalize_adapthist(data)        
        patientData.append(data)
    for i in range(negLen):
        data = readScan(i, "NegTest")
        labels.append(0)
        data = exposure.equalize_adapthist(data)        
        patientData.append(data)
    
    patientData = numpy.stack(patientData).astype(float)
    return patientData, numpy.stack(labels)
    

In [ ]:
dirName='/Users/MOOSE/Desktop/DL/CT_Project/'
imageCount(dirName)